## IMS mismatch with M1M3 motion system - SITCOM-760

In [1]:
# Define time frame to retrieve data in the EFD database
t_start = "2023-05-30T04:35:00"
t_end = "2023-05-30T05:30:00"

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools as itt
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from astropy import units as u
from astropy.time import Time, TimezoneInfo

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

In [4]:
# Create client to access the EFD database
client = vandv.efd.create_efd_client()

In [5]:
pos_columns = ["xPosition", "yPosition", "zPosition"]

In [6]:
# get IMS data
df_ims = await client.select_time_series(
    "lsst.sal.MTM1M3.imsData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_ims = df_ims.set_index("private_rcvStamp")
df_ims.index = pd.to_datetime(df_ims.index, unit="s")

# get position columns and convert to mm
df_ims = df_ims[pos_columns] * 1e3

In [7]:
# get Hard Point state

df_HPState = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_hardpointActuatorState", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPState = df_HPState.set_index("private_rcvStamp")
df_HPState.index = pd.to_datetime(df_HPState.index, unit="s")

In [8]:
# get Hard Point Data
df_HPData = await client.select_time_series(
    "lsst.sal.MTM1M3.hardpointActuatorData", 
    "*", 
    Time(t_start, format="isot", scale="utc"),
    Time(t_end, format="isot", scale="utc"), 
)

df_HPData = df_HPData.set_index("private_rcvStamp")
df_HPData.index = pd.to_datetime(df_HPData.index, unit="s")

# get position columns and convert to mm
df_HPData = df_HPData[pos_columns] * 1e3

In [9]:
# Find all the time windows during which the HP is in Standy state (stable)  
mask_HP_stable = (df_HPState[[f"motionState{j}" for j in range(1, 6)]] == 0).all(axis=1) 

delta_t = pd.Timedelta(0.5, "seconds")
time_stable = []

for i, mask in enumerate(mask_HP_stable):
    if mask:
        if i+1 < len(mask_HP_stable) and not mask_HP_stable[mask_HP_stable.index[i+1]]:
            t0 = mask_HP_stable.index[i]
            
# Remove a small time interval at the end of the time window in order to make sure that HP is in Standy state for any ids timestamp 
            t1 = mask_HP_stable.index[i+1] - delta_t
            time_stable.append([t0, t1])

In [10]:
# Store results in a dataframe in order to plot them later
results = pd.DataFrame(columns=["ids_x_mean", "ids_y_mean", "ids_z_mean", "HP_x_mean", "HP_y_mean", "HP_z_mean", 
                                "offset_x_mean", "offset_y_mean", "offset_z_mean"])

for index, time_window in enumerate(time_stable):
    t0 = time_window[0]
    t1 = time_window[1]
    ids_x_mean = np.mean(df_ims[t0:t1]['xPosition'])
    ids_y_mean = np.mean(df_ims[t0:t1]['yPosition'])
    ids_z_mean = np.mean(df_ims[t0:t1]['zPosition'])
    HP_x_mean = np.mean(df_HPData[t0:t1]['xPosition'])
    HP_y_mean = np.mean(df_HPData[t0:t1]['yPosition'])
    HP_z_mean = np.mean(df_HPData[t0:t1]['zPosition'])
    offset_x_mean = ids_x_mean - HP_x_mean
    offset_y_mean = ids_y_mean - HP_y_mean
    offset_z_mean = ids_z_mean - HP_z_mean
    
    # Store results in dataframe
    row = pd.DataFrame({"ids_x_mean":ids_x_mean, "ids_y_mean":ids_y_mean, "ids_z_mean":ids_y_mean, 
                        "HP_x_mean":HP_x_mean, "HP_y_mean":HP_y_mean, "HP_z_mean":HP_z_mean,
                        "offset_x_mean":offset_x_mean, "offset_y_mean":offset_y_mean, "offset_z_mean": offset_z_mean}, index=[index])
    results = pd.concat([results, row])

axes = [ "x", "y", "z"]
sigma = {}
mean = {}

for axis in axes:
    sigma[axis] = np.std(results["offset_" + axis + "_mean"])
    mean[axis] = np.mean(results["offset_" + axis + "_mean"])

In [11]:
results

,ids_x_mean,ids_y_mean,ids_z_mean,HP_x_mean,HP_y_mean,HP_z_mean,offset_x_mean,offset_y_mean,offset_z_mean
0,-1.461109,5.512146,5.512146,0.000000,0.000000e+00,0.000000,-1.461109,5.512146,-1.388209
1,-0.473468,5.471183,5.471183,0.999919,-9.500117e-07,-0.000188,-1.473387,5.471184,-1.393196
2,-2.449521,5.520565,5.520565,-0.999919,9.500752e-07,-0.000188,-1.449602,5.520564,-1.396737
3,-1.479338,6.499647,6.499647,0.000002,1.000036e+00,-0.000249,-1.479339,5.499611,-1.398133
4,-1.457433,4.521586,4.521586,-0.000002,-1.000036e+00,-0.000126,-1.457432,5.521622,-1.394173
...,...,...,...,...,...,...,...,...,...
92,-1.453090,4.511495,4.511495,-0.000005,-9.998180e-01,-1.000172,-1.453085,5.511313,-1.390350
93,-1.465042,5.506173,5.506173,0.000001,-3.389537e-05,-0.000248,-1.465044,5.506207,-1.382007
94,-1.463042,5.508030,5.508030,0.000007,-7.477081e-05,-0.000065,-1.463049,5.508105,-1.381122
95,-1.460582,5.496491,5.496491,0.000172,-1.079703e-04,-0.000125,-1.460754,5.496599,-1.380899


In [ ]:
# Plot results
locator = mdates.AutoDateLocator(minticks=3, maxticks=3)
formatter = mdates.ConciseDateFormatter(locator)

title = "SITCOM-760"
fig, axs = plt.subplots(num=title, nrows=2, ncols=3, figsize=(10, 8))

axes = [ "x", "y", "z"]
for i in range(3):
    max = np.max(results[f"offset_{axes[i]}_mean"])
    min = np.min(results[f"offset_{axes[i]}_mean"])
    axs[0][i].hist(results[f"offset_{axes[i]}_mean"], bins=50, range=[min-0.1*abs(min), max+0.1*abs(max)])
    axs[0][i].set_xlabel(f"Offset (ids-HP) {axes[i]} axis (mm)")
    val_mean = mean[axes[i]]
    val_sigma = sigma[axes[i]]
    axs[0][i].set_title("mean = %.2f mm \n sigma = %.2e mm" %(val_mean, val_sigma))
    
    axs[1][i].plot(df_ims.index, df_ims[axes[i] + "Position"], label="ims")
    axs[1][i].plot(df_HPData.index, df_HPData[axes[i] + "Position"], label = "HP")
    axs[1][i].xaxis.set_major_formatter(formatter)
    max = np.max([np.max(df_ims[axes[i] + "Position"]), np.max(df_HPData[axes[i] + "Position"])])
    min = np.min([np.min(df_ims[axes[i] + "Position"]), np.min(df_HPData[axes[i] + "Position"])])
    axs[1][i].set_ylim([min-0.2, max+0.2])
    axs[1][i].set_xlabel("Time")
    axs[1][i].set_ylabel(f"{axes[i]} Position (mm)")
    axs[1][i].legend()
    
fig.suptitle(title + "\n")
fig.tight_layout()
plt.subplots_adjust(top=0.90)